<a href="https://colab.research.google.com/github/DarioRugg/KickLearning/blob/main/feature_analysis/Text_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
![ ! -f "translate.py" ] && wget translate.py https://raw.githubusercontent.com/DarioRugg/KickLearning/main/feature_analysis/translate.py
!pip install pycld2 sentencepiece transformers polyglot pyicu pySBD

In [3]:
filename = 'file_0000_scraped.csv'

In [ ]:
for i in range(9):
  filename = f'file_000{i}_scraped.csv'
  !python translate.py $filename

using device: cuda:0
Filtering of other languages done in 24.5 seconds
Translating from language es, 7 more languages to do. 
Elapsed time: 0.41 minutes
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
2021-06-25 17:54:07.059028: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
